In [18]:
import os, re
os.chdir('/mnt/ufs18/home-230/pandavis/cellular_resilience/CellResilienceModel/codebase/Banksy_py')

import numpy as np
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings("ignore") 

import pandas as pd
import pickle

import scipy.sparse as sparse
from scipy.io import mmread
from scipy.stats import pearsonr, pointbiserialr

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D

import seaborn as sns
import scanpy as sc


import random
seed = 42
np.random.seed(seed)
random.seed(seed)

In [19]:
pwd

'/mnt/ufs18/home-230/pandavis/cellular_resilience/CellResilienceModel/codebase/Banksy_py'

In [22]:
adata = sc.read_10x_h5('../../data/BreastCancer10xGenomics_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_cell_feature_matrix.h5')
adata

AnnData object with n_obs × n_vars = 167782 × 313
    var: 'gene_ids', 'feature_types', 'genome'

In [23]:
#adata.to_df().to_csv('Xenium_FFPE_Human_Breast_Cancer_Rep1_cell_feature.csv')

In [24]:
#!git clone https://github.com/prabhakarlab/Banksy_py.git

In [25]:
adata.to_df().head()

,ABCC11,ACTA2,ACTG2,ADAM9,ADGRE5,ADH1B,ADIPOQ,AGR3,AHSP,AIF1,...,TUBB2B,TYROBP,UCP1,USP53,VOPP1,VWF,WARS,ZEB1,ZEB2,ZNF562
1,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,6.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [26]:
#cell_data = pd.read_parquet('../../data/BreastCancer10xGenomics_Rep1/cells.parquet')
cell_data = pd.read_csv('../../data/BreastCancer10xGenomics_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_cells.csv', index_col=0)
#cell_data.set_index(cell_data.columns[0], inplace=True)
cell_data.index = cell_data.index.astype(str)
cell_data.head(3)

,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,total_counts,cell_area,nucleus_area
cell_id,,,,,,,,
1,377.663005,843.541888,154,0,0,154,110.361875,45.562656
2,382.078658,858.944818,64,0,0,64,87.919219,24.248906
3,319.839529,869.196542,57,0,0,57,52.561875,23.526406


In [27]:
cell_annotations = pd.read_csv('../../data/BreastCancer10xGenomics_Rep1/exported_data/BreastCancer10XGenomics_Rep1_cells_annotated.csv', index_col=0)
cell_annotations.index = cell_annotations.index.astype(str)
cell_annotations.head(3)

,seurat_clusters,celltype
1,4,Macrophages
2,1,T cells
3,1,T cells


In [28]:
cell_index = list(adata.obs.index.astype(str))

In [29]:
cell_data = cell_data.loc[cell_index, :]
cell_annotations = cell_annotations.loc[cell_index,:]

In [30]:
adata.obs['celltype'] = cell_annotations['celltype']
adata.obs[['x_centroid', 'y_centroid']] = cell_data[['x_centroid', 'y_centroid']]
adata.obsm['coord_xy'] = np.array(adata.obs[['x_centroid', 'y_centroid']])

In [34]:
adata

AnnData object with n_obs × n_vars = 156831 × 313
    obs: 'celltype', 'x_centroid', 'y_centroid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    obsm: 'coord_xy'

In [31]:
from banksy_utils.load_data import load_adata, display_adata
from banksy_utils.filter_utils import filter_cells
from banksy_utils.filter_utils import normalize_total, filter_hvg, print_max_min
from banksy.main import median_dist_to_nearest_neighbour
from banksy.main import median_dist_to_nearest_neighbour
from banksy.initialize_banksy import initialize_banksy
from banksy.embed_banksy import generate_banksy_matrix
from banksy.main import concatenate_all
from banksy_utils.umap_pca import pca_umap
from banksy.cluster_methods import run_Leiden_partition

In [32]:
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("MT-")

sc.pp.calculate_qc_metrics(
    adata,
    qc_vars=["mt"],      
    log1p=True,          
    inplace=True,      
    percent_top=[10, 50, 100]  
)

In [33]:
adata = filter_cells(adata, 
             min_count=40, 
             max_count=1000, 
             MT_filter=20, 
             gene_filter=10)

Cells before filtering: 167782

Cells after count filter: 156831

cells after MT filter: 156831

Genes after minimum cells per gene filter: 313



In [35]:
adata = normalize_total(adata)

adata, adata_allgenes = filter_hvg(adata,
            n_top_genes = 300,
            flavor="seurat")

--- Max-Min before normalization -----
Displaying max and min of Dataset
Max: 170.0, Min: 0.0

--- Max-Min after normalization -----
Displaying max and min of Dataset
Max: 76.86363220214844, Min: 0.0

--- Normalized and log-transformed data -----
Displaying max and min of Dataset
Max: 4.354959011077881, Min: 0.0

Displaying dataset after filtering by HVG
View of AnnData object with n_obs × n_vars = 156831 × 300
    obs: 'celltype', 'x_centroid', 'y_centroid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    obsm: 'coord_xy'


In [36]:
coord_keys = ('x_centroid', 'y_centroid', 'coord_xy')

plot_graph_weights = True
k_geom = 15 # only for fixed type
max_m = 1 # azumithal transform up to kth order
nbr_weight_decay = "scaled_gaussian" # can also be "reciprocal", "uniform" or "ranked"

# Find median distance to closest neighbours, the median distance will be `sigma`
nbrs = median_dist_to_nearest_neighbour(adata, key = coord_keys[2])


Median distance to closest cell = 9.68733903271435

---- Ran median_dist_to_nearest_neighbour in 0.55 s ----



In [37]:
banksy_dict = initialize_banksy(
    adata,
    coord_keys,
    k_geom,
    nbr_weight_decay=nbr_weight_decay,
    max_m=max_m,
    plt_edge_hist=False,
    plt_nbr_weights=False,
    plt_agf_angles=False, # takes long time to plot
    plt_theta=False,
)


Median distance to closest cell = 9.68733903271435

---- Ran median_dist_to_nearest_neighbour in 0.54 s ----

---- Ran generate_spatial_distance_graph in 1.16 s ----

---- Ran row_normalize in 0.51 s ----

---- Ran generate_spatial_weights_fixed_nbrs in 5.92 s ----

---- Ran generate_spatial_distance_graph in 1.77 s ----

---- Ran theta_from_spatial_graph in 2.67 s ----

---- Ran row_normalize in 0.52 s ----

---- Ran generate_spatial_weights_fixed_nbrs in 9.97 s ----



In [38]:
resolutions = [0.7] # clustering resolution for UMAP
pca_dims = [20] # Dimensionality in which PCA reduces to
lambda_list = [0.2] # list of lambda parameters

banksy_dict, banksy_matrix = generate_banksy_matrix(adata, banksy_dict, lambda_list, max_m)

Runtime Jan-03-2025-12-16

300 genes to be analysed:
Gene List:
Index(['ABCC11', 'ACTA2', 'ACTG2', 'ADAM9', 'ADGRE5', 'ADH1B', 'ADIPOQ',
       'AGR3', 'AHSP', 'AIF1',
       ...
       'TUBB2B', 'TYROBP', 'UCP1', 'USP53', 'VOPP1', 'VWF', 'WARS', 'ZEB1',
       'ZEB2', 'ZNF562'],
      dtype='object', length=300)

Check if X contains only finite (non-NAN) values
Decay Type: scaled_gaussian
Weights Object: {'weights': {0: <156831x156831 sparse matrix of type '<class 'numpy.float64'>'
	with 2352465 stored elements in Compressed Sparse Row format>, 1: <156831x156831 sparse matrix of type '<class 'numpy.complex128'>'
	with 4704930 stored elements in Compressed Sparse Row format>}}

Nbr matrix | Mean: 0.57 | Std: 1.44
Size of Nbr | Shape: (156831, 300)
Top 3 entries of Nbr Mat:

[[0.         2.98229855 1.93571754]
 [0.24420866 4.06143617 2.0743679 ]
 [0.05468251 1.94259636 2.05182258]]

AGF matrix | Mean: 0.17 | Std: 0.31
Size of AGF mat (m = 1) | Shape: (156831, 300)
Top entries of AGF:
[[

In [39]:
with open("../../data/BreastCancer10xGenomics_Rep1/exported_data/banksy_dict.pkl", "wb") as f:
    pickle.dump(banksy_dict, f)

banksy_matrix.write('../../data/BreastCancer10xGenomics_Rep1/exported_data/banksy_matrix.h5ad')

In [19]:
banksy_dict["nonspatial"] = {
    0.0: {"adata": concatenate_all([adata.X], 0, adata=adata), }
}

print(banksy_dict['nonspatial'][0.0]['adata'])

Scale factors squared: [1.]
Scale factors: [1.]
AnnData object with n_obs × n_vars = 537180 × 300
    obs: 'celltype', 'x_centroid', 'y_centroid', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_10_genes', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'is_nbr', 'k'


In [ ]:
pca_umap(banksy_dict,
         pca_dims = pca_dims,
         add_umap = True,
         plt_remaining_var = False,
         )

Current decay types: ['scaled_gaussian', 'nonspatial']

Reducing dims of dataset in (Index = scaled_gaussian, lambda = 0.2)

Setting the total number of PC = 20
Original shape of matrix: (537180, 900)
Reduced shape of matrix: (537180, 20)
------------------------------------------------------------
min_value = -27.728579701585243, mean = 5.271650919259342e-16, max = 59.229836345794375

Conducting UMAP and adding embeddings to adata.obsm["reduced_pc_20_umap"]


In [ ]:
results_df, max_num_labels = run_Leiden_partition(
    banksy_dict,
    resolutions,
    num_nn = 50,
    num_iterations = -1,
    partition_seed = seed,
    match_labels = True,
)